# Extract data

## Connect to DB

In [59]:
import pandas as pd
import psycopg2

CONNECT_DB = "host=127.0.0.1 port=5432 dbname=postgres user=postgres password=password123"

## Look at the characteristics of the dataset

In [55]:
df =pd.read_csv('banksim_artificial.csv')

In [9]:
df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [10]:
df.dtypes

step             int64
customer        object
age             object
gender          object
zipcodeOri      object
merchant        object
zipMerchant     object
category        object
amount         float64
fraud            int64
dtype: object

## -Create Table

In [15]:
import psycopg2

create_table_query = '''CREATE TABLE banksim_artificial (
    step INT,
    customer VARCHAR,
    age VARCHAR,
    gender VARCHAR,
    zipcodeOri VARCHAR,
    merchant VARCHAR,
    zipMerchant VARCHAR,
    category VARCHAR,
    amount NUMERIC,
    fraud INT
);'''

try:
    # Make connection to db
    cxn = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()
    
    # Send sql query to request
    cur.execute(create_table_query)
    cxn.commit()  # Commit the transaction
    
    print("Table 'banksim_artificial' created successfully.")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)
    
finally:
    # Closing database connection.
    if cxn:
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")

Table 'banksim_artificial' created successfully.
PostgreSQL connection is closed


## -Insert dataset to database which was successfully created previously

In [16]:
try:
    # Make connection to db
    cxn = psycopg2.connect(CONNECT_DB)
    
    # Create a cursor to db
    cur = cxn.cursor()
    
    with open('./banksim_artificial.csv', 'r') as f:
        # skip first row, header row
        next(f)
        cur.copy_from(f, 'banksim_artificial', sep=",")
        cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")
        print("banksim_artificial table populated")

PostgreSQL connection is closed
banksim_artificial table populated


# Transform

## -Transform data with pandas

### Select new data from database

In [35]:
import psycopg2

# Make connection to db
CONNECT_DB = "host=127.0.0.1 port=5432 dbname=postgres user=postgres password=password123"
cxn = psycopg2.connect(CONNECT_DB)

# Create a cursor to db
cur = cxn.cursor()

# Query to select all data
select_all = 'SELECT * FROM banksim_artificial limit 10;'

# Execute the query
cur.execute(select_all)

# Fetch all rows after executing the query
rows = cur.fetchall()

# Print all rows
for row in rows:
    print(row)

print("Select all data successfully")

(4, "'C1634492186'", "'3'", "'M'", "'28007'", "'M348934600'", "'28007'", "'es_transportation'", Decimal('40.88'), 0)
(4, "'C1857263556'", "'1'", "'F'", "'28007'", "'M348934600'", "'28007'", "'es_transportation'", Decimal('4.57'), 0)
(4, "'C629808856'", "'3'", "'M'", "'28007'", "'M348934600'", "'28007'", "'es_transportation'", Decimal('41.57'), 0)
(4, "'C717345438'", "'3'", "'M'", "'28007'", "'M1823072687'", "'28007'", "'es_transportation'", Decimal('61.43'), 0)
(4, "'C718801730'", "'2'", "'M'", "'28007'", "'M348934600'", "'28007'", "'es_transportation'", Decimal('64.86'), 0)
(4, "'C251602943'", "'2'", "'F'", "'28007'", "'M348934600'", "'28007'", "'es_transportation'", Decimal('18.63'), 0)
(4, "'C47809392'", "'3'", "'M'", "'28007'", "'M1198415165'", "'28007'", "'es_wellnessandbeauty'", Decimal('158.83'), 1)
(4, "'C1490048317'", "'6'", "'F'", "'28007'", "'M1198415165'", "'28007'", "'es_wellnessandbeauty'", Decimal('541.86'), 1)
(4, "'C1633477199'", "'2'", "'M'", "'28007'", "'M348934600'"

### update database and cleansing for make good database

In [62]:
import psycopg2

# Your database connection details
CONNECT_DB = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'password123',
    'host': '127.0.0.1',
    'port': '5432'
}

# Update query
update_query = """
    UPDATE banksim_artificial
    SET 
        customer = REPLACE(customer, '''', ''),
        age = REPLACE(age, '''', ''),
        gender = REPLACE(gender, '''', ''),
        zipcodeOri = REPLACE(zipcodeOri, '''', ''),
        merchant = REPLACE(merchant, '''', ''),
        zipMerchant = REPLACE(zipMerchant, '''', ''),
        category = REPLACE(category, '''', '');
"""

try:
    # Make connection to db
    cxn = psycopg2.connect(**CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()
    
    # Send sql query to request
    cur.execute(update_query)
    cxn.commit()  # Commit the transaction
    
    print("Data in 'banksim_artificial' updated successfully.")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)
    
finally:
    # Closing database connection.
    if cxn:
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")


Data in 'banksim_artificial' updated successfully.
PostgreSQL connection is closed


In [64]:
import psycopg2

# Make connection to db
CONNECT_DB = "host=127.0.0.1 port=5432 dbname=postgres user=postgres password=password123"
cxn = psycopg2.connect(CONNECT_DB)

# Create a cursor to db
cur = cxn.cursor()

# Query to select all data
select_all = 'SELECT * FROM banksim_artificial limit 10;'

# Execute the query
cur.execute(select_all)

# Fetch all rows after executing the query
rows = cur.fetchall()

# Print all rows
for row in rows:
    print(row)

print("Select all data successfully")

(4, 'C435572486', '1', 'F', '28007', 'M480139044', '28007', 'es_health', Decimal('118.15'), 0)
(5, 'C150587559', '2', 'F', '28007', 'M480139044', '28007', 'es_health', Decimal('238.14'), 1)
(5, 'C23992005', '3', 'M', '28007', 'M547558035', '28007', 'es_fashion', Decimal('100.03'), 0)
(5, 'C2007043031', '5', 'F', '28007', 'M85975013', '28007', 'es_food', Decimal('67.46'), 0)
(5, 'C875898611', '3', 'F', '28007', 'M840466850', '28007', 'es_tech', Decimal('149.01'), 0)
(5, 'C1048588440', '6', 'M', '28007', 'M85975013', '28007', 'es_food', Decimal('7.72'), 0)
(5, 'C1051682954', '2', 'F', '28007', 'M348934600', '28007', 'es_transportation', Decimal('38.68'), 0)
(5, 'C1956971543', '1', 'F', '28007', 'M348934600', '28007', 'es_transportation', Decimal('29.47'), 0)
(6, 'C1034685704', '2', 'F', '28007', 'M50039827', '28007', 'es_health', Decimal('78.59'), 0)
(5, 'C2095429384', '2', 'F', '28007', 'M348934600', '28007', 'es_transportation', Decimal('5.83'), 0)
Select all data successfully


In [74]:
import psycopg2

# Your database connection details
CONNECT_DB = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'password123',
    'host': '127.0.0.1',
    'port': '5432'
}

# Update query
update_query = """
    ALTER TABLE banksim_artificial
        ALTER COLUMN step TYPE SMALLINT,
        ALTER COLUMN customer TYPE VARCHAR(20),
        ALTER COLUMN gender TYPE VARCHAR(1),
        ALTER COLUMN zipcodeOri TYPE VARCHAR(10),
        ALTER COLUMN merchant TYPE VARCHAR(20),
        ALTER COLUMN zipMerchant TYPE VARCHAR(10),
        ALTER COLUMN category TYPE VARCHAR(50),
        ALTER COLUMN amount TYPE FLOAT,
        ALTER COLUMN fraud TYPE SMALLINT;
"""

try:
    # Make connection to db
    cxn = psycopg2.connect(**CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()
    
    # Send sql query to request
    cur.execute(update_query)
    cxn.commit()  # Commit the transaction
    
    print("Data types in 'banksim_artificial' table updated successfully.")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)
    
finally:
    # Closing database connection.
    if cxn:
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")


Data types in 'banksim_artificial' table updated successfully.
PostgreSQL connection is closed


In [75]:
import psycopg2

# Make connection to db
CONNECT_DB = "host=127.0.0.1 port=5432 dbname=postgres user=postgres password=password123"
cxn = psycopg2.connect(CONNECT_DB)

# Create a cursor to db
cur = cxn.cursor()

# Query to select all data
select_all = 'SELECT * FROM banksim_artificial limit 10;'

# Execute the query
cur.execute(select_all)

# Fetch all rows after executing the query
rows = cur.fetchall()

# Print all rows
for row in rows:
    print(row)

print("Select all data successfully")

(6, 'C42779194', '6', 'F', '28007', 'M348934600', '28007', 'es_transportation', 43.65, 0)
(48, 'C914000857', 'U', 'E', '28007', 'M85975013', '28007', 'es_food', 52.77, 0)
(6, 'C679458582', '4', 'F', '28007', 'M348934600', '28007', 'es_transportation', 56.79, 0)
(6, 'C332727422', '5', 'M', '28007', 'M1823072687', '28007', 'es_transportation', 3.99, 0)
(6, 'C1807970113', '4', 'F', '28007', 'M348934600', '28007', 'es_transportation', 8.24, 0)
(6, 'C775533998', '2', 'F', '28007', 'M348934600', '28007', 'es_transportation', 22.87, 0)
(6, 'C851156881', '1', 'M', '28007', 'M348934600', '28007', 'es_transportation', 22.35, 0)
(6, 'C41461366', '3', 'F', '28007', 'M348934600', '28007', 'es_transportation', 23.19, 0)
(6, 'C407219337', '3', 'M', '28007', 'M348934600', '28007', 'es_transportation', 11.17, 0)
(6, 'C936853935', '3', 'M', '28007', 'M1823072687', '28007', 'es_transportation', 26.68, 0)
Select all data successfully


### Menghitung berapa rata rata jumlah di age dan mengcek apakah ada data yang aneh

In [79]:
import psycopg2

# Make connection to db
CONNECT_DB = "host=127.0.0.1 port=5432 dbname=postgres user=postgres password=password123"
cxn = psycopg2.connect(CONNECT_DB)

# Create a cursor to db
cur = cxn.cursor()

# Query to select all data
selectage_query = """
        SELECT age, count(age)
            FROM public.banksim_artificial
            group by age;
"""

# Execute the query
cur.execute(selectage_query)

# Fetch all rows after executing the query
rows = cur.fetchall()

# Print all rows
for row in rows:
    print(row)

print("Select all data successfully")

('0', 2452)
('1', 58131)
('2', 187310)
('3', 147131)
('4', 109025)
('5', 62642)
('6', 26774)
('U', 1178)
Select all data successfully


### mengubah data age U menjadi 2

In [80]:
import psycopg2

# Your database connection details
CONNECT_DB = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'password123',
    'host': '127.0.0.1',
    'port': '5432'
}

# Update query
update_query = """
    UPDATE banksim_artificial
    SET age = '2'
    WHERE age = 'U';
"""

try:
    # Make connection to db
    cxn = psycopg2.connect(**CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()
    
    # Send sql query to request
    cur.execute(update_query)
    cxn.commit()  # Commit the transaction
    
    print("conver U to 2 in 'banksim_artificial' table updated successfully.")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)
    
finally:
    # Closing database connection.
    if cxn:
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")

conver U to 2 in 'banksim_artificial' table updated successfully.
PostgreSQL connection is closed


### Jika data sudah oke finalisasi dengan mengubah type data yang sebelumnya salah dengan tepat

In [94]:
import psycopg2

# Your database connection details
CONNECT_DB = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'password123',
    'host': '127.0.0.1',
    'port': '5432'
}

# Update query
update_query = """
   ALTER TABLE banksim_artificial
        ALTER COLUMN customer TYPE VARCHAR(255),
        ALTER COLUMN merchant TYPE VARCHAR(255),
        ALTER COLUMN category TYPE VARCHAR(255)
"""

try:
    # Make connection to db
    cxn = psycopg2.connect(**CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()
    
    # Send sql query to request
    cur.execute(update_query)
    cxn.commit()  # Commit the transaction
    
    print("Data types in 'banksim_artificial' table updated successfully.")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)
    
finally:
    # Closing database connection.
    if cxn:
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")


Data types in 'banksim_artificial' table updated successfully.
PostgreSQL connection is closed


In [107]:
import psycopg2

# Your database connection details
CONNECT_DB = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'password123',
    'host': '127.0.0.1',
    'port': '5432'
}

# Update query
update_query = """
        ALTER TABLE banksim_artificial
            ALTER COLUMN zipcodeOri TYPE integer USING zipcodeOri::integer,
            ALTER COLUMN zipMerchant TYPE integer USING zipMerchant::integer;
"""

try:
    # Make connection to db
    cxn = psycopg2.connect(**CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()
    
    # Send sql query to request
    cur.execute(update_query)
    cxn.commit()  # Commit the transaction
    
    print("Data types zipcodeOri and zipMerchant in 'banksim_artificial' table updated successfully.")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)
    
finally:
    # Closing database connection.
    if cxn:
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")


Data types zipcodeOri and zipMerchant in 'banksim_artificial' table updated successfully.
PostgreSQL connection is closed


### update data type from column age to smallint

In [108]:
import psycopg2

# Your database connection details
CONNECT_DB = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'password123',
    'host': '127.0.0.1',
    'port': '5432'
}

# Update query
update_query = """
ALTER TABLE banksim_artificial
        ALTER COLUMN age TYPE SMALLINT USING CASE WHEN age = '0' THEN 0
                                                    WHEN age = '1' THEN 1
                                                    WHEN age = '2' THEN 2
                                                    WHEN age = '3' THEN 3
                                                    WHEN age = '4' THEN 4
                                                    WHEN age = '5' THEN 5
                                                    WHEN age = '6' THEN 6
                                                    ELSE NULL END
"""

try:
    # Make connection to db
    cxn = psycopg2.connect(**CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()
    
    # Send sql query to request
    cur.execute(update_query)
    cxn.commit()  # Commit the transaction
    
    print("Data age to smallint in 'banksim_artificial' table updated successfully.")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)
    
finally:
    # Closing database connection.
    if cxn:
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")


Data age to smallint in 'banksim_artificial' table updated successfully.
PostgreSQL connection is closed


### pandas data anlyst

In [114]:
def pandas_db_server_fetch(sql_query):
    cxn = None
    try:
        # Make connection to db
        cxn = psycopg2.connect(**CONNECT_DB)

        # Send sql query to request and create dataframe
        df = pd.read_sql(sql_query, cxn)

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL:", error)
        df = None

    finally:
        # Closing database connection.
        if cxn:
            cxn.close()
            print("PostgreSQL connection is closed")
        return df

In [118]:
select_query = '''SELECT * FROM banksim_artificial'''

data_df = pandas_db_server_fetch(select_query)
data_df.head()

C:\Users\FHKHASAN\AppData\Local\Temp\ipykernel_8240\3670625772.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, cxn)


PostgreSQL connection is closed


,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
0,6,C42779194,6,F,28007,M348934600,28007,es_transportation,43.65,0
1,6,C679458582,4,F,28007,M348934600,28007,es_transportation,56.79,0
2,6,C332727422,5,M,28007,M1823072687,28007,es_transportation,3.99,0
3,6,C1807970113,4,F,28007,M348934600,28007,es_transportation,8.24,0
4,6,C775533998,2,F,28007,M348934600,28007,es_transportation,22.87,0


In [119]:
data_df.describe()

,step,age,zipcodeori,zipmerchant,amount,fraud
count,594643.000000,594643.000000,594643.0,594643.0,594643.000000,594643.000000
mean,94.986827,3.004246,28007.0,28007.0,37.890135,0.012108
std,51.053632,1.324784,0.0,0.0,111.402831,0.109369
min,0.000000,0.000000,28007.0,28007.0,0.000000,0.000000
25%,52.000000,2.000000,28007.0,28007.0,13.740000,0.000000
50%,97.000000,3.000000,28007.0,28007.0,26.900000,0.000000
75%,139.000000,4.000000,28007.0,28007.0,42.540000,0.000000
max,179.000000,6.000000,28007.0,28007.0,8329.960000,1.000000


In [121]:
data_df.dtypes

step             int64
customer        object
age              int64
gender          object
zipcodeori       int64
merchant        object
zipmerchant      int64
category        object
amount         float64
fraud            int64
dtype: object

In [124]:
data_df.isna()

,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
594638,False,False,False,False,False,False,False,False,False,False
594639,False,False,False,False,False,False,False,False,False,False
594640,False,False,False,False,False,False,False,False,False,False
594641,False,False,False,False,False,False,False,False,False,False


In [125]:
data_df.isna().sum()

step           0
customer       0
age            0
gender         0
zipcodeori     0
merchant       0
zipmerchant    0
category       0
amount         0
fraud          0
dtype: int64

# Load

## -Migrating PostgreSQL Data from Local to Google Cloud SQL

In [127]:
from google.cloud import storage

### make storage bucket gcs

In [152]:
service_account_file = r'S:\Data Fellowship\code\Weekly_Assignment\key_week1.json'

In [135]:
def create_gcs_bucket(project_id, bucket_name, service_account_file):
    from google.cloud import storage

    """
    project_id (str): Your Google Cloud Platform (GCP) project ID.
    bucket_name (str): Name of the new bucket to be created.
    service_account_file (str): Your service account JSON key file
    """
    
    try:
        # Initialize a GCS client
        client = storage.Client.from_service_account_json(service_account_file)

        # Create a new bucket
        bucket = client.create_bucket(bucket_name)

        print(f'Bucket {bucket_name} created successfully.')
    
    except Exception as e:
        print(f'Error creating bucket: {str(e)}')


### upload file to gcs

In [136]:
create_gcs_bucket('iconic-indexer-418610', 'weekly-assignment1', service_account_file)

Bucket weekly-assignment1 created successfully.


In [154]:
def upload_sql_to_gcs(bucket_name, local_sql_file_path, remote_gcs_file_name):
    # Initialize the GCS client
    storage_client = storage.Client.from_service_account_json(service_account_file)

    # Get the GCS bucket
    bucket = storage_client.get_bucket(bucket_name)

    # Create a blob (remote file) in the bucket
    blob = bucket.blob(remote_gcs_file_name)

    # Upload the local SQL file to GCS
    blob.upload_from_filename(local_sql_file_path)

    print(f"SQL file '{local_sql_file_path}' uploaded to GCS as '{remote_gcs_file_name}'.")

# Example usage
bucket_name = 'weekly-assignment1'
local_sql_file_path = r'S:\Data Fellowship\code\Weekly_Assignment\banksim_artificial_table.sql'
remote_gcs_file_name = 'banksim_artificial.sql'

upload_sql_to_gcs(bucket_name, local_sql_file_path, remote_gcs_file_name)

SQL file 'S:\Data Fellowship\code\Weekly_Assignment\banksim_artificial_table.sql' uploaded to GCS as 'banksim_artificial.sql'.


# Proses selanjutnya untuk proses di cloud akan dijelaskan di dokumentasi pada:

[Link to Weekly Assignment 5 section](https://seed-waitress-434.notion.site/Weekly-Assignment-5-bbcd21d9fec04b50881fcd3b3c43607f?pvs=4)
